This dataset comprises 1 million professionally annotated article-summary pairs from BBC, extracted using a set of carefully designed heuristics.

In [4]:
!git clone https://github.com/AzzaAbdelGhani/xl-sum.git

Cloning into 'xl-sum'...
remote: Enumerating objects: 1148, done.
remote: Counting objects: 100% (1148/1148), done.
remote: Compressing objects: 100% (817/817), done.
remote: Total 1148 (delta 314), reused 1124 (delta 301), pack-reused 0
Receiving objects: 100% (1148/1148), 5.51 MiB | 10.45 MiB/s, done.
Resolving deltas: 100% (314/314), done.


In [5]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:29
🔁 Restarting kernel...


In [6]:
!cd xl-sum/seq2seq
!conda create python==3.7.9 pytorch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 cudatoolkit=10.2 -c pytorch -p ./env
!conda activate ./env # or source activate ./env (for older versions of anaconda)
!bash xl-sum/seq2seq/setup.sh 

Solving environment: - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /content/env

  added / updated specs:
    - cudatoolkit=10.2
    - python==3.7.9
    - pytorch==1.7.1
    - torchaudio==0.7.2
    - torchvision==0.8.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    ca-certificates-2021.10.8  |       ha878542_0         1

In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD" -O XLSum_complete_v2.0.tar.bz2 && rm -rf /tmp/cookies.txt
!tar -xjvf XLSum_complete_v2.0.tar.bz2
!python xl-sum/seq2seq/extract_data.py -i XLSum_complete_v2.0/ -o XLSum_input/

--2022-03-29 19:26:04--  https://docs.google.com/uc?export=download&confirm=t&id=1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD
Resolving docs.google.com (docs.google.com)... 142.250.141.100, 142.250.141.102, 142.250.141.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.141.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/apjh66soiqelr542kq9kp45i5h9ckr95/1648581900000/07441260459817291408/*/1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD?e=download [following]
--2022-03-29 19:26:04--  https://doc-08-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/apjh66soiqelr542kq9kp45i5h9ckr95/1648581900000/07441260459817291408/*/1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD?e=download
Resolving doc-08-24-docs.googleusercontent.com (doc-08-24-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to doc-08-24-docs.googleusercontent.com (doc-

In [2]:
!pip install jsonlines

     |████████████████████████████████| 60 kB 5.3 MB/s 


In [3]:
import jsonlines 
import pandas as pd 
import random

languages = ['arabic','english','french','spanish', 'indonesian','turkish','ukrainian','hindi']
train_list = [] #493753
test_list = [] #40461
val_list = [] #40391
for w in ['train', 'test','val']: 
  for lang in languages :
    with jsonlines.open('XLSum_complete_v2.0/{lang}_{w}.jsonl'.format(lang=lang, w=w)) as f:
      for line in f.iter():
        if len(line['text']) > 1000:
          if w == 'train' : 
            train_list.append(line)
          elif w == 'test':
            test_list.append(line)
          else:
            val_list.append(line)
random.shuffle(train_list)
random.shuffle(test_list)
random.shuffle(val_list)
train_dataset = pd.DataFrame(train_list)
test_dataset = pd.DataFrame(test_list)
val_dataset = pd.DataFrame(val_list)

In [4]:
len(val_dataset)

40391

In [5]:
len(train_dataset)

493753

In [6]:
len(test_dataset)

40461

In [ ]:
train_dataset.to_csv('train_dataset.csv')
test_dataset.to_csv('test_dataset.csv')
val_dataset.to_csv('val_dataset.csv')
from google.colab import drive
drive.mount('/content/gdrive')
!cp train_dataset.csv "gdrive/My Drive/Dataset"
!cp test_dataset.csv "gdrive/My Drive/Dataset"
!cp val_dataset.csv "gdrive/My Drive/Dataset"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train_dataset.head()

,id,url,title,summary,text
0,160609_social_media_streisand_effect_tk,https://www.bbc.com/hindi/india/2016/06/160609...,"जितना दबाएंगे, ख़बर उतनी ही बड़ी होगी",पिछले दिनों मुंबई पुलिस ने फ़ेसबुक और यूट्यूब ...,उन्हें राजनीतिक पार्टियों के नेताओं की ओर से श...
1,business-51548361,https://www.bbc.com/news/business-51548361,Climate change: Fertiliser could be used to po...,Ocean-going ships could be powered by ammonia ...,By Roger HarrabinBBC environment analyst The c...
2,160501_iraq_protests_moqtada_al_sadr_md,https://www.bbc.com/hindi/international/2016/0...,इराक़: शिया प्रदर्शनकारियों पर कार्रवाई का आदेश,इराक़ के प्रधानमंत्री हैदर-अल-अबादी ने शनिवार ...,प्रधानमंत्री ने कहा है कि जिन लोगों ने पुलिस प...
3,uk-wales-politics-35354209,https://www.bbc.com/news/uk-wales-politics-353...,Tory MPs warn chancellor of tax devolution opp...,Some Welsh Tory MPs have warned the chancellor...,By James WilliamsBBC Wales political correspon...
4,world-latin-america-19546909,https://www.bbc.com/news/world-latin-america-1...,Guyana country profile,"Sandwiched between Venezuela and Suriname, the...","Since independence from the UK in 1966, Guyana..."


In [ ]:
test_dataset.head()

,id,url,title,summary,text
0,uk-politics-18170333,https://www.bbc.com/news/uk-politics-18170333,David Cameron and Ed Miliband clash on sackings,Prime Minister David Cameron has clashed with ...,Venture capitalist Adrian Beecroft said in a r...
1,sinhala.070911_tax_protest,https://www.bbc.com/sinhala/news/story/2007/09...,Protests against tax bills launched,The opposition parties in Sri Lanka have vowed...,Main opposition and a breakaway faction of the...
2,151223_eeuu_clinton_abuela_ilm,https://www.bbc.com/mundo/noticias/2015/12/151...,#Notmyabuela: el revés mediático de Hillary Cl...,El más reciente intento de la campaña de Hilla...,El último intento de Hillary Clinton por llega...
3,business-21454912,https://www.bbc.com/news/business-21454912,Japanese economy contracts for third straight ...,Japan's economy contracted for the third strai...,The economy shrank 0.1% compared with the prev...
4,india-51598821,https://www.bbc.com/hindi/india-51598821,शाहीन बाग़: सड़क का एक हिस्सा खोलने को लेकर अस...,दिल्ली के शाहीन बाग़ में नागरिकता संशोधन क़ानू...,अभी तक यह स्पष्ट नहीं हो पाया है कि इस सड़क को...


In [ ]:
val_dataset.head()